In [1]:
import re
import os
import argparse
import datetime
from types import SimpleNamespace

import torch
from torch import nn
import torch.optim as optim
import numpy as np
import gymnasium as gym

from tianshou.data import Collector, Batch, to_torch
from tianshou.data.types import RolloutBatchProtocol
from tianshou.data.buffer.vecbuf import VectorReplayBuffer, ReplayBuffer
from tianshou.env import SubprocVectorEnv
from tianshou.policy import RBVEPolicy, BasePolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import ActorProb, Critic
from examples.offline.utils import load_buffer_d4rl
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
from tianshou.trainer import OffpolicyTrainer
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 262, in register_gymnasium_envs
    _register_dm_control_envs()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 26, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shim

In [2]:
device = "cuda:3"

In [3]:
def parse_value(value):
    # Convert simple types (int, float, bool, None)
    if value.isdigit():
        return int(value)
    elif re.match(r'^\d+\.\d+$', value):
        return float(value)
    elif value == "True":
        return True
    elif value == "False":
        return False
    elif value == "None":
        return None
    elif value.startswith("[") and value.endswith("]"):
        # Convert the list items
        items = re.split(r',(?=[^\]]*(?:\[|$))', value[1:-1])
        return [parse_value(item.strip()) for item in items]
    elif value.startswith("(") and value.endswith(")"):
        # Convert the tuple items
        items = re.split(r',(?=[^\)]*(?:\(|$))', value[1:-1])
        # Special case for single-item tuple
        if len(items) == 2 and items[0].strip() != '':
            return (parse_value(items[0].strip()),)
        return tuple(parse_value(item.strip()) for item in items)
    elif value.startswith("'") and value.endswith("'"):
        return value[1:-1]
    # Else, return the value as-is
    return value

def get_args(event_file):
    ea = EventAccumulator(event_file)
    ea.Reload()  # Load the file
    # Get the text data
    texts = ea.Tags()["tensors"]
    # Extract the actual text content
    text_data = {}
    for tag in texts:
        events = ea.Tensors(tag)
        for event in events:
            # You can extract the wall_time and step if needed
            # wall_time, step, value = event.wall_time, event.step, event.text
            text_data[tag] = event.tensor_proto.string_val
    data = text_data['args/text_summary'][0]
    # Convert bytes to string
    data_str = data.decode('utf-8')
    # Remove the "Namespace(" prefix and the trailing ")"
    data_str = data_str[len("Namespace("):-1]
    # Split into key-value pairs
    key_values = re.split(r',(?=\s\w+=)', data_str)
    # Parse each key-value pair
    args_dict = {}
    for kv in key_values:
        key, value = kv.split('=', 1)
        key = key.strip()
        args_dict[key] = parse_value(value)
    args = SimpleNamespace(**args_dict)
    return args

In [4]:
log_path = "/data/user/R901105/dev/log/Hopper-v2/rbve/1/240502-103556"
files = os.listdir(log_path)
event_file = [f for f in files if f.startswith('event')][0]
full_path = os.path.join(log_path, event_file)
args = get_args(full_path)
args

namespace(task='Hopper-v2',
          seed=1,
          expert_data_task='hopper-medium-v2',
          ac_path='/data/user/R901105/dev/log/Hopper-v2/action_classification/0/240427-225051/model.pt',
          ac_hidden_sizes=[512, 512, 512],
          buffer_size=1000000,
          hidden_sizes=[256, 256, 256],
          actor_lr=0.0001,
          critic_lr=0.0003,
          q_marge=5.0,
          alpha='(-3, tensor([0.]',
          device='cuda:1',
          requires_grad='True), Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0\n))',
          auto_alpha=True,
          alpha_lr=0.0001,
          cql_alpha_lr=0.0003,
          start_timesteps=10000,
          epoch=1000,
          step_per_epoch=5000,
          batch_size=256,
          norm_layer=True,
          tau=0.005,
          temperature=1.0,
      

In [5]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(MyModel, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.input_norm = nn.LayerNorm(hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_sizes[i], hidden_sizes[i+1])
            for i in range(len(hidden_sizes) - 1)
        ])
        self.hidden_norms = nn.ModuleList([
            nn.LayerNorm(hidden_sizes[i+1])
            for i in range(len(hidden_sizes) - 1)
        ])
        self.output_layer = nn.Linear(hidden_sizes[-1], 1)  # 1 output for energy score
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.input_norm(x)
        x = torch.relu(x)
        for hidden_layer, hidden_norm in zip(self.hidden_layers, self.hidden_norms):
            x = hidden_layer(x)
            x = hidden_norm(x)
            x = torch.relu(x)
        output = self.output_layer(x)
        return torch.sigmoid(output)

In [6]:
def load_policy(path, alpha):
    env = gym.make(args.task)

    # action classifier
    id_model = MyModel(args.state_dim + args.action_dim, args.ac_hidden_sizes).to(device)
    id_model.load_state_dict(torch.load(args.ac_path))
    print("Loaded AC model from:", args.ac_path)

    # actor network
    net_a = Net(args.state_shape, hidden_sizes=args.hidden_sizes, device=device)
    actor = ActorProb(
        net_a,
        args.action_shape,
        device=device,
        unbounded=True,
        conditioned_sigma=True,
    ).to(device)
    actor_optim = torch.optim.Adam(actor.parameters(), lr=args.actor_lr)

    # critic network
    net_c1 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    net_c2 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    critic1 = Critic(net_c1, device=device).to(device)
    critic1_optim = torch.optim.Adam(critic1.parameters(), lr=args.critic_lr)
    critic2 = Critic(net_c2, device=device).to(device)
    critic2_optim = torch.optim.Adam(critic2.parameters(), lr=args.critic_lr)

    if args.auto_alpha:
        target_entropy = -np.prod(env.action_space.shape)
        log_alpha = torch.tensor([np.log(alpha)], requires_grad=True, device=device, dtype=torch.float32)
        alpha_optim = torch.optim.Adam([log_alpha], lr=args.alpha_lr)
        args.alpha = (target_entropy, log_alpha, alpha_optim)

    policy = RBVEPolicy(
        actor,
        actor_optim,
        critic1,
        critic1_optim,
        critic2,
        critic2_optim,
        id_model=id_model,
        action_space=env.action_space,
        cql_alpha_lr=args.cql_alpha_lr,
        cql_weight=args.cql_weight,
        tau=args.tau,
        gamma=args.gamma,
        alpha=args.alpha,
        temperature=args.temperature,
        with_lagrange=args.with_lagrange,
        lagrange_threshold=args.lagrange_threshold,
        min_action=np.min(env.action_space.low),
        max_action=np.max(env.action_space.high),
        calibrated=args.calibrated,
        q_marge=args.q_marge,
        device=device,
    )

    # load policy
    policy.load_state_dict(torch.load(path, map_location=device))
    print("Loaded agent from: ", path)

    return policy

In [7]:
offline_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/rbve/0/240430-111233/policy.pth", alpha=0.014)

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


Loaded AC model from: /data/user/R901105/dev/log/Hopper-v2/action_classification/0/240427-225051/model.pt
Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/rbve/0/240430-111233/policy.pth


In [8]:
offline_data = load_buffer_d4rl(args.expert_data_task)

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile:   0%|          | 0/21 [00:00<?, ?it/s]

load datafile: 100%|██████████| 21/21 [00:01<00:00, 17.08it/s]


In [9]:
env = gym.make(args.task)

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [10]:
class OnlineRBVEPolicy(BasePolicy):
    def __init__(self, policy: RBVEPolicy, action_space):
        super().__init__(action_space=action_space, action_scaling=True)
        self.policy = policy
        self.action_classifier = policy.id_model
        self.ac_loss_fn = nn.BCELoss()
        self.ac_optimiser = optim.SGD(self.action_classifier.parameters(), lr=1e-3)

    def train(self, mode: bool = True) -> "OnlineRBVEPolicy":
        self.policy.train(mode)
        return self
    
    def forward(self, batch: RolloutBatchProtocol, state=None, **kwargs):
        return self.policy(batch, state, **kwargs)
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.policy.process_fn(batch, buffer, indices)

    def get_negatives(self, batch: RolloutBatchProtocol):
        rand_actions = np.random.uniform(low=env.action_space.low, high=env.action_space.high, size=(len(batch), env.action_space.shape[0])).astype(np.float32) 
        return torch.cat([batch.obs, torch.tensor(rand_actions, dtype=torch.float32, device=device)], dim=1)   

    def train_ac(self, batch: RolloutBatchProtocol):
        batch = to_torch(batch, dtype=torch.float32, device=device)
        # positive samples
        X = torch.cat([batch.obs, batch.act], dim=1)
        y = torch.ones(len(batch), 1).to(device)
        # negative samples
        X_neg = self.get_negatives(batch)
        y_neg = torch.zeros(len(batch), 1).to(device)
        # combine and shuffle
        X = torch.cat([X, X_neg], dim=0)
        y = torch.cat([y, y_neg], dim=0)
        idx = torch.randperm(len(X))
        X = X[idx]
        y = y[idx]
        pred = self.action_classifier(X)
        loss = self.ac_loss_fn(pred, y)
        self.ac_optimiser.zero_grad()
        loss.backward()
        self.ac_optimiser.step()
        return loss.item()

    def learn(self, batch, **kwargs):
        info = self.policy.learn(batch)
        info["ac_loss"] = self.train_ac(batch)
        return info

In [11]:
policy = OnlineRBVEPolicy(offline_policy, env.action_space)

In [12]:
test_buffer = ReplayBuffer(5000)
train_collector = Collector(policy, env, offline_data)
test_collector = Collector(policy, env, test_buffer)

/data/user/R901105/dev/my_fork/tianshou/tianshou/data/collector.py:70: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [13]:
test_collector.collect(n_episode=5)

{'n/ep': 5,
 'n/st': 4288,
 'rews': array([3239.2962205 , 2347.45773982, 3240.77453363, 3238.72680753,
        1899.92545403]),
 'lens': array([1000,  706, 1000, 1000,  582]),
 'idxs': array([   0, 1000, 1706, 2706, 3706]),
 'rew': 2793.2361511012828,
 'len': 857.6,
 'rew_std': 564.7024894183991,
 'len_std': 178.75748935359323}

In [14]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = os.path.join(args.task, "onlinerbve", now)
log_path = os.path.join("../../log", log_name)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

In [15]:
from tianshou.trainer import OffpolicyTrainer

result = OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=200,
    step_per_epoch=500,
    step_per_collect=1,
    episode_per_test=1,
    batch_size=512,
    logger=logger,
    update_per_step=1,
    test_in_train=False,
).run()

Epoch #1: 501it [00:20, 24.34it/s, ac_loss=0.144, alpha=0.015, dataset_in_dist=467.151, env_step=500, gradient_step=500, in_dist=4183.034, len=0, loss/actor=-250.128, loss/alpha=1.340, loss/cql1=0.434, loss/cql2=0.436, loss/critic1=0.959, loss/critic2=0.975, n/ep=0, n/st=1, q_dataset=250.192, rew=0.00]                          


Epoch #1: test_reward: 3335.694768 ± 0.000000, best_reward: 3335.694768 ± 0.000000 in #1


Epoch #2: 501it [00:20, 24.37it/s, ac_loss=0.144, alpha=0.015, dataset_in_dist=467.155, env_step=1000, gradient_step=1000, in_dist=4153.963, len=0, loss/actor=-250.263, loss/alpha=1.346, loss/cql1=0.448, loss/cql2=0.452, loss/critic1=0.967, loss/critic2=0.951, n/ep=0, n/st=1, q_dataset=250.302, rew=0.00]                         


Epoch #2: test_reward: 1062.016183 ± 0.000000, best_reward: 3335.694768 ± 0.000000 in #1


Epoch #3: 501it [00:20, 24.17it/s, ac_loss=0.144, alpha=0.016, dataset_in_dist=467.362, env_step=1500, gradient_step=1500, in_dist=4125.892, len=0, loss/actor=-250.679, loss/alpha=1.372, loss/cql1=0.460, loss/cql2=0.464, loss/critic1=0.914, loss/critic2=0.948, n/ep=0, n/st=1, q_dataset=250.718, rew=0.00]                         


...

Epoch #198: test_reward: 3370.706546 ± 0.000000, best_reward: 3495.840031 ± 0.000000 in #93


Epoch #199: 501it [00:19, 26.04it/s, ac_loss=0.147, alpha=0.021, dataset_in_dist=466.691, env_step=99500, gradient_step=99500, in_dist=4034.594, len=26128, loss/actor=-272.369, loss/alpha=0.079, loss/cql1=0.567, loss/cql2=0.567, loss/critic1=1.417, loss/critic2=1.428, n/ep=0, n/st=1, q_dataset=272.333, rew=85252.16]                          


Epoch #199: test_reward: 3364.718145 ± 0.000000, best_reward: 3495.840031 ± 0.000000 in #93


Epoch #200: 501it [00:19, 25.94it/s, ac_loss=0.147, alpha=0.021, dataset_in_dist=466.922, env_step=100000, gradient_step=100000, in_dist=4006.097, len=26128, loss/actor=-272.560, loss/alpha=-0.058, loss/cql1=0.590, loss/cql2=0.589, loss/critic1=1.979, loss/critic2=1.972, n/ep=0, n/st=1, q_dataset=272.519, rew=85252.16]                         


Epoch #200: test_reward: 3358.332979 ± 0.000000, best_reward: 3495.840031 ± 0.000000 in #93
